In [3]:
#pyTorch

import azureml.core
print(azureml.core.VERSION)
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

from azureml.core import Environment

curated_env = Environment.get(workspace=ws, name="abi_pytorch2")



ModuleNotFoundError: No module named 'azureml'

In [4]:
%pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 40.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 20.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.7 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 3.7 MB/s eta 0:00:000:00:0100:01

In [1]:
import numpy
import pandas
import matplotlib.pyplot
import csv
import torch
import math
import torch.nn as nn

Load data, make into tensors...

In [3]:
schools = pandas.read_csv("/home/azureuser/cloudfiles/code/Users/ariley/Data/Schools/School_Counts/NE_school_density_10km.csv")

#schools = schools.iloc[:, 3:]

schools = schools.dropna()

schools.head()

,fid,id,left,top,right,bottom,NUMPOINTS
0,1,1,-4.885870e+06,-129741.256537,-4.875870e+06,-139741.256537,0
1,2,2,-4.885870e+06,-139741.256537,-4.875870e+06,-149741.256537,0
2,3,3,-4.885870e+06,-149741.256537,-4.875870e+06,-159741.256537,0
3,4,4,-4.885870e+06,-159741.256537,-4.875870e+06,-169741.256537,0
4,5,5,-4.885870e+06,-169741.256537,-4.875870e+06,-179741.256537,0


In [4]:
schools.NUMPOINTS.describe()

count    37275.000000
mean         2.892770
std         16.122163
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max       1187.000000
Name: NUMPOINTS, dtype: float64

In [5]:
mu_ = schools.NUMPOINTS.mean()
sigma_ = numpy.sqrt(schools.NUMPOINTS.var())

https://github.com/meaneych/PyTorch_CountDistribution_Examples/blob/main/Poisson_PyTorch_Autograd.ipynb

In [6]:
x = torch.autograd.Variable(torch.from_numpy(schools.to_numpy())).type(torch.FloatTensor)
l_mu = torch.autograd.Variable(torch.rand(1), requires_grad=True) 

Poisson Loss Functions

In [7]:
def poisson_nll(x, log_mu):
    nll = -torch.sum(-torch.exp(log_mu) + x*torch.log(torch.exp(log_mu)) - torch.lgamma(x))
    return nll

In [ ]:
def poissonLoss(predicted, observed):
    """Custom loss function for Poisson model."""
    loss = torch.mean(predicted-observed*torch.log(predicted))
    return loss

pyTorch ZIP loss function????

mixture loss function??? possible?

In [ ]:
def poisson_nll(x, log_mu):
    if x == 0:
        nll = 
    nll = -torch.sum(-torch.exp(log_mu) + x*torch.log(torch.exp(log_mu)) - torch.lgamma(x))
    return nll

From ChatGPT

In [34]:
class ZeroInflatedPoissonLoss(torch.nn.Module):
    def __init__(self):
        super(ZeroInflatedPoissonLoss, self).__init__()

    def forward(self, y_true, pi, lambda_param):
        """
        Args:
            y_true (torch.Tensor): True target values (ground truth).
            pi (torch.Tensor): Probability of excess zeros (0 <= pi <= 1).
            lambda_param (torch.Tensor): Poisson distribution parameter.
        """
        # Calculate the probability mass function for each data point
        prob_zero = pi + (1 - pi) * torch.exp(-lambda_param)
        prob_non_zero = (1 - pi) * torch.exp(-lambda_param) * lambda_param**y_true / torch.factorial(y_true)

        # Combine the probabilities for zero and non-zero values
        prob_combined = torch.where(y_true == 0, prob_zero, prob_non_zero)

        # Calculate the negative log-likelihood
        loss = -torch.log(prob_combined)

        # Take the mean across the batch
        nll = torch.mean(loss)

        return nll

In [8]:
## Learning rate
learning_rate = 2e-4

## Training loop
for t in range(100):
    ## Backprop on negative log likelihood loss
    loss = nn.PoissonNLLLoss()
    NLLp = loss(l_mu, x)
    NLLp.backward()
    ## Logging to console
    if t % 10 == 0:
        print("Iteration = ", t, 
              "loglik  =", NLLp.data.numpy(), 
              "l_mu =", l_mu.data.numpy(), 
              "dL/dlmu = ", l_mu.grad.data.numpy())
    ## SGD update of parms
    l_mu.data -= learning_rate * l_mu.grad.data
    ## Zero the gradients
    l_mu.grad.data.zero_()

Iteration =  0 loglik  = 724479.7 l_mu = [0.48885423] dL/dlmu =  [1481993.5]
Iteration =  10 loglik  = -4391876600.0 l_mu = [-2963.4956] dL/dlmu =  [1481991.9]
Iteration =  20 loglik  = -8784477000.0 l_mu = [-5927.48] dL/dlmu =  [1481991.9]
Iteration =  30 loglik  = -13177078000.0 l_mu = [-8891.464] dL/dlmu =  [1481991.9]
Iteration =  40 loglik  = -17569680000.0 l_mu = [-11855.448] dL/dlmu =  [1481991.9]
Iteration =  50 loglik  = -21962279000.0 l_mu = [-14819.433] dL/dlmu =  [1481991.9]
Iteration =  60 loglik  = -26354880000.0 l_mu = [-17783.416] dL/dlmu =  [1481991.9]
Iteration =  70 loglik  = -30747480000.0 l_mu = [-20747.4] dL/dlmu =  [1481991.9]
Iteration =  80 loglik  = -35140080000.0 l_mu = [-23711.385] dL/dlmu =  [1481991.9]
Iteration =  90 loglik  = -39532683000.0 l_mu = [-26675.37] dL/dlmu =  [1481991.9]


In [9]:
## Final estimate of Poisson mean parm
[l_mu.data.numpy(), numpy.exp(l_mu.data.numpy())]

[array([-29639.354], dtype=float32), array([0.], dtype=float32)]

Now let's try with just urban maybe?

In [12]:
schools = pandas.read_csv("/home/azureuser/cloudfiles/code/Users/ariley/Data/Schools/School_Counts/NE_school_density_10km.csv")
schools = schools.iloc[:, 3:]
schools = schools.dropna()
#schools = schools[schools["smod"].isin([21,22,23,30])]
schools = schools[schools["smod"] == 30]

schools.head()
#matplotlib.pyplot.hist(schools.NUMPOINTS)

KeyError: 'smod'

In [53]:
mu_ = schools.NUMPOINTS.mean()
sigma_ = numpy.sqrt(schools.NUMPOINTS.var())

x = torch.autograd.Variable(torch.from_numpy(schools.to_numpy())).type(torch.FloatTensor)
l_mu = torch.autograd.Variable(torch.rand(1), requires_grad=True) 

## Learning rate
learning_rate = 2e-4

## Training loop
for t in range(100):
    ## Backprop on negative log likelihood loss
    loss = nn.PoissonNLLLoss()
    NLLp = loss(l_mu, x)
    NLLp.backward()
    ## Logging to console
    if t % 10 == 0:
        print("Iteration = ", t, 
              "loglik  =", NLLp.data.numpy(), 
              "l_mu =", l_mu.data.numpy(), 
              "dL/dlmu = ", l_mu.grad.data.numpy())
    ## SGD update of parms
    l_mu.data -= learning_rate * l_mu.grad.data
    ## Zero the gradients
    l_mu.grad.data.zero_()

Iteration =  0 loglik  = 30012.994 l_mu = [0.31442636] dL/dlmu =  [95450.17]
Iteration =  10 loglik  = -18190966.0 l_mu = [-190.58344] dL/dlmu =  [95448.805]
Iteration =  20 loglik  = -36411908.0 l_mu = [-381.48096] dL/dlmu =  [95448.805]
Iteration =  30 loglik  = -54632860.0 l_mu = [-572.3786] dL/dlmu =  [95448.805]
Iteration =  40 loglik  = -72853840.0 l_mu = [-763.2764] dL/dlmu =  [95448.805]
Iteration =  50 loglik  = -91074780.0 l_mu = [-954.17426] dL/dlmu =  [95448.805]
Iteration =  60 loglik  = -109295736.0 l_mu = [-1145.0717] dL/dlmu =  [95448.805]
Iteration =  70 loglik  = -127516610.0 l_mu = [-1335.9689] dL/dlmu =  [95448.805]
Iteration =  80 loglik  = -145737580.0 l_mu = [-1526.8661] dL/dlmu =  [95448.805]
Iteration =  90 loglik  = -163958380.0 l_mu = [-1717.7633] dL/dlmu =  [95448.805]


In [54]:
## Final estimate of Poisson mean parm
[l_mu.data.numpy(), numpy.exp(l_mu.data.numpy())]

[array([-1908.6605], dtype=float32), array([0.], dtype=float32)]